In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py

In [37]:
temp_country = pd.read_csv('data/bad_global_land_temperatures_by_country.csv')
temp_world = pd.read_csv('data/bad_global_temperatures_world.csv')


In [38]:
temp_world

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3187,2015-08-01,14.755,0.072,20.699,0.110,9.005,0.170,17.589,0.057
3188,2015-09-01,12.999,0.079,18.845,0.088,7.199,0.229,17.049,0.058
3189,2015-10-01,10.801,0.102,16.450,0.059,5.232,0.115,16.290,0.062
3190,2015-11-01,7.433,0.119,12.892,0.093,2.157,0.106,15.252,0.063


In [39]:
#Removing the duplicated countries and countries for which no information about the temperature
temp_country_clean = temp_country[~temp_country['Country'].isin(
    ['Denmark', 'Antarctica', 'France', 'Europe', 'Netherlands', 'United Kingdom', 'Africa', 'South America'])]
temp_country_clean = temp_country_clean.replace(
    ['Denmark (Europe)', 'France (Europe)', 'Netherlands (Europe)', 'United Kingdom (Europe)'],
    ['Denmark', 'France', 'Netherlands', 'United Kingdom'])

In [40]:
temp_country_clean['dt'] = pd.to_datetime(temp_country_clean['dt'])
temp_world['dt'] = pd.to_datetime(temp_world['dt'])


In [41]:
start_date = pd.to_datetime('1900-1-1')
end_date = pd.to_datetime('2013-1-1')
mask = temp_country_clean['dt'] >= start_date
mask_world = temp_world['dt'] >= start_date
temp_country_1900 = temp_country_clean.loc[mask]
temp_world_1900 = temp_world.loc[mask]
temp_world_1900

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
1874,1906-03-01,4.866,0.200,11.008,0.264,-1.229,0.353,14.245,0.117
1875,1906-04-01,8.626,0.242,14.786,0.581,2.422,0.383,15.230,0.122
1876,1906-05-01,11.240,0.290,17.146,0.437,4.922,0.426,15.769,0.132
1877,1906-06-01,13.334,0.289,19.287,0.323,7.275,0.321,16.436,0.127
1878,1906-07-01,14.122,0.323,20.058,0.391,8.186,0.345,16.662,0.137
...,...,...,...,...,...,...,...,...,...
3187,2015-08-01,14.755,0.072,20.699,0.110,9.005,0.170,17.589,0.057
3188,2015-09-01,12.999,0.079,18.845,0.088,7.199,0.229,17.049,0.058
3189,2015-10-01,10.801,0.102,16.450,0.059,5.232,0.115,16.290,0.062
3190,2015-11-01,7.433,0.119,12.892,0.093,2.157,0.106,15.252,0.063


In [66]:
from pathlib import Path

temp_country_yearly = temp_country_1900.groupby([pd.Grouper(key='dt', freq='Y'), 'Country'])[
    'AverageTemperature'].mean().reset_index(name='Year')
temp_world_yearly = temp_world_1900.groupby([pd.Grouper(key='dt', freq='Y')])[
    'LandAverageTemperature'].mean().reset_index(name='Year')
temp_country_yearly.rename(columns={'dt': 'Year', 'Year': 'AverageTemperature'}, inplace=True)
temp_country_yearly['Year'] = pd.DatetimeIndex(temp_country_yearly['Year']).year
temp_world_yearly.rename(columns={'dt': 'Year', 'Year': 'AverageTemperature'}, inplace=True)
temp_world_yearly['Year'] = pd.DatetimeIndex(temp_world_yearly['Year']).year
temp_world_yearly['Country'] = 'World'
final_temp_df = pd.concat([temp_world_yearly, temp_country_yearly], sort=True)
final_temp_df.sort_values(['Year', 'Country'], inplace=True)
filepath = Path('./test/data.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
final_temp_df.reset_index(inplace=True)
final_temp_df.to_csv(filepath)


In [43]:
countries = np.unique(temp_country_yearly[temp_country_yearly['Country'] != 'World']['Country'])
temp_dif_lst = []
for country in countries:
    country_temp = temp_country_yearly[(temp_country_yearly['Country'] == country) & (
            (temp_country_yearly['Year'] == 2013) | (temp_country_yearly['Year'] == 1900))][
        'AverageTemperature'].diff()
    country_temp.drop(index=country_temp.index[0], axis=0,
                      inplace=True)
    temp_dif_country = country_temp.values
    if len(temp_dif_country) > 0:
        temp_dif_lst.append(temp_dif_country[0])


In [44]:
data = [dict(type='choropleth', locations=countries, z=temp_dif_lst, locationmode='country names', text=countries,
             marker=dict(line=dict(color='rgb(0,0,0)', width=1)),
             colorbar=dict(autotick=True, tickprefix='', title='# Temperature \nDifferences,\n°C'))]
layout = dict(title='Temperature differences in countries (1900 - 2013)',
              geo=dict(showframe=False, showocean=True, oceancolor='rgb(0,255,255)',
                       projection=dict(type='orthographic', rotation=dict(lon=60, lat=10), ),
                       lonaxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)'),
                       lataxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)')), )
fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldmap')

In [45]:
emission = pd.read_csv('data/bad_historical_emission_by_country.csv')
emission

,Country,Data source,Sector,Gas,Unit,2018,2017,2016,2015,2014,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,World,CAIT,Total including LUCF,CO2,MtCO₂e,36441.55,35588.70,35160.60,34521.91,34558.59,...,24957.30,24895.32,25292.92,24214.92,23890.22,23260.29,23124.00,22988.29,23056.40,22849.92
1,World,CAIT,Total excluding LUCF,CO2,MtCO₂e,35248.74,34542.44,34081.00,34038.49,34098.75,...,23398.99,23240.47,23082.12,22694.41,22215.88,21591.78,21455.52,21319.81,21387.92,21181.44
2,World,CAIT,Energy,CO2,MtCO₂e,33746.74,33070.44,32620.00,32608.49,32627.75,...,22714.09,22584.68,22423.32,22054.72,21589.48,20999.68,20899.32,20783.81,20873.62,20680.14
3,World,CAIT,Electricity/Heat,CO2,MtCO₂e,15590.95,15167.36,14949.51,15009.83,15232.52,...,10134.66,10080.66,9853.19,9586.39,9231.22,9042.50,8945.80,8894.60,8735.04,8596.17
4,China,CAIT,Total excluding LUCF,CO2,MtCO₂e,10313.46,10017.77,9814.31,9830.43,9894.94,...,3149.20,3232.12,3128.65,3064.88,3084.87,2763.90,2643.53,2418.18,2302.19,2173.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,Saint Vincent and the Grenadines,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2303,Vanuatu,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2304,Samoa,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2305,Zambia,CAIT,Fugitive Emissions,CO2,MtCO₂e,NaN,NaN,NaN,NaN,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [57]:
emission['Mean'] = emission.iloc[:, 5:34].mean(axis=1).round(decimals=3)
total_emission = emission[emission['Sector'] == 'Total including LUCF']
total_emission

,Country,Data source,Sector,Gas,Unit,2018,2017,2016,2015,2014,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,Mean
0,World,CAIT,Total including LUCF,CO2,MtCO₂e,36441.55,35588.70,35160.60,34521.91,34558.59,...,24895.32,25292.92,24214.92,23890.22,23260.29,23124.00,22988.29,23056.40,22849.92,28930.456
5,China,CAIT,Total including LUCF,CO2,MtCO₂e,9663.36,9367.67,9164.21,9120.27,9184.77,...,2882.75,2779.27,2715.50,2735.48,2414.50,2294.12,2068.77,1952.78,1823.96,5405.082
12,United States,CAIT,Total including LUCF,CO2,MtCO₂e,4749.57,4581.90,4656.84,4563.52,4683.35,...,5172.06,5129.29,4864.46,4708.31,4654.52,4581.76,4461.62,4389.50,4426.40,4900.768
17,European Union (27),CAIT,Total including LUCF,CO2,MtCO₂e,2636.99,2692.12,2669.54,2321.61,2263.78,...,3134.03,3142.23,3210.93,3113.68,3060.09,3072.40,3133.06,3247.48,3286.44,2946.370
19,India,CAIT,Total including LUCF,CO2,MtCO₂e,2400.25,2267.16,2149.01,2085.38,2072.03,...,618.73,600.38,555.60,519.98,466.79,431.31,409.09,386.17,341.32,1192.656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,Nauru,CAIT,Total including LUCF,CO2,MtCO₂e,0.07,0.07,0.06,0.05,0.05,...,0.10,0.10,0.10,0.11,0.11,0.11,0.12,0.13,0.13,0.076
1934,Tuvalu,CAIT,Total including LUCF,CO2,MtCO₂e,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.009
2041,Niue,CAIT,Total including LUCF,CO2,MtCO₂e,0.00,0.00,0.00,0.00,0.00,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.007
2125,Fiji,CAIT,Total including LUCF,CO2,MtCO₂e,-0.65,-0.73,-0.81,-1.03,-1.24,...,-1.76,-1.74,-1.72,-1.78,-1.79,-1.79,-1.78,-1.81,-1.81,-1.454


In [58]:
countries_emission = np.unique(total_emission[total_emission['Country'] != 'World']['Country'])
mean_emission_lst = []
for country in countries_emission:
    mean_total_emission = total_emission[(total_emission['Country'] == country)]['Mean']
    if len(mean_total_emission.values) > 0:
        mean_emission_lst.append(mean_total_emission.values[0])

In [59]:
emission_countries_yearly = total_emission.melt(id_vars=["Country"], value_vars=total_emission.iloc[:, 5:34],
                                                var_name="Year", value_name="TotalEmission")

In [60]:
data_emission = [
    dict(type='choropleth', locations=countries_emission, z=mean_emission_lst, locationmode='country names',
         text='Mean emission',
         marker=dict(line=dict(color='rgb(0,0,0)', width=1)),
         colorbar=dict(autotick=True, tickprefix='', title='Mean\nEmission,\n(MtCO₂e)'))]
layout = dict(title='Mean CO2 emission in countries (1990 - 2018)',
              geo=dict(showframe=False, showocean=True, oceancolor='rgb(0,255,255)',
                       projection=dict(type='orthographic', rotation=dict(lon=60, lat=10), ),
                       lonaxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)'),
                       lataxis=dict(showgrid=True, gridcolor='rgb(102, 102, 102)')), )
fig_emission = dict(data=data_emission, layout=layout)
py.iplot(fig_emission, validate=False, filename='worldmap')


In [63]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

world = final_temp_df[final_temp_df.Country == "World"]
world_emission = emission_countries_yearly[emission_countries_yearly["Country"] == "World"]

fig.add_trace(go.Scatter(x=world.Year, y=world.AverageTemperature, name="Average Temperature"), secondary_y=False)

fig.add_trace(go.Scatter(x=world_emission.Year, y=world_emission.TotalEmission, name="Total Emission"), secondary_y=True)

fig.update_layout(title_text="Average Temperature and Total Emission of the World")

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Temperature (Celcius)", secondary_y=False)
fig.update_yaxes(title_text="Total Emission (MTCO2)", secondary_y=True)

fig.show()

In [51]:
clean_temperature_df = pd.read_csv('data/clean/clean_global_land_temperature_by_country.csv',
                                   usecols=[i for i in range(1, 4)])
is_NaN = clean_temperature_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_have_NaN = clean_temperature_df[row_has_NaN]

In [52]:
clean_emission_df = pd.read_csv('data/clean/clean_total_emission_by_country.csv', usecols=[i for i in range(1,4)])
clean_emission_df.head()


,Country,Year,TotalEmission
0,World,2018,36441.55
1,China,2018,9663.36
2,United States,2018,4749.57
3,European Union (27),2018,2636.99
4,India,2018,2400.25
